In [16]:
import glob
import os
import treeswift

In [166]:


def compute_parsimony_cost(tree, leaf_label_df, states):

    drop_leaves = set()
    for n in laml_tree.traverse_leaves():
        cell_idx = int(n.label.split('_')[1][4:])
        # print(cell_idx)
        if cell_idx not in set(leaf_label_df['cell_idx'].values):
            drop_leaves.add(n.label)

    cost_dict = dict()
    idx = 0
    for n in tree.traverse_postorder(): 
        # print(n.label,f'node_{idx}' )
        n.quiet_idx = f'node_{idx}'
        idx += 1

    for n in tree.traverse_postorder(): 
        #print("node.label", n.label, "node.quiet_idx", n.quiet_idx)
        if n.is_leaf():
            if n.label in drop_leaves:
                cost_dict[n.quiet_idx] = dict()
                for state in states:
                    cost_dict[n.quiet_idx][state] = 0
            else: 
                node_idx = int(n.label.split('_')[1][4:])
                cost_dict[n.quiet_idx] = dict()
                idx = leaf_label_df.index[leaf_label_df['cell_idx'] == node_idx].tolist()
                #print("lookup idx", idx)
                obs_state = leaf_label_df.loc[idx, 'state_idx'].values[0]
                #print("obs_state", obs_state)
                for state in states:
                    if obs_state == state:
                        cost_dict[n.quiet_idx][state] = 0
                    else:
                        cost_dict[n.quiet_idx][state] = 1
        else:
            # print('internal node')
            cost_dict[n.quiet_idx] = dict()
            # print('num_children', len(n.children))
            for u in n.children:
                for state in states:
                    if state not in cost_dict[n.quiet_idx]:
                        cost_dict[n.quiet_idx][state] = 0

                    argmin_state = min(cost_dict[u.quiet_idx], key=cost_dict[u.quiet_idx].get)
                    cost = cost_dict[u.quiet_idx][argmin_state]
                    # print("argmin", argmin_state, "cost", cost)
                    if state == argmin_state: 
                        cost_dict[n.quiet_idx][state] += cost
                    else:
                        cost_dict[n.quiet_idx][state] += cost + 1


    #return cost_dict[tree.root.label][min(cost_dict[tree.root.label], key=cost_dict[tree.root.label].get)]
    return cost_dict[tree.root.quiet_idx][min(cost_dict[tree.root.quiet_idx], key=cost_dict[tree.root.quiet_idx].get)]




In [33]:
import pandas as pd

states_folder = "/Users/gc3045/laml2_experiments/BaseMEM_Magic/Magic/Figures3_5_and_SupFigures2_4-6/lineage_analysis/all_position_barcodes"
state_files = glob.glob(os.path.join(states_folder, "*states.txt"))

all_df = dict()
# read in gene fluorescence measurements
for g in state_files:
    # print(g)
    pos = os.path.basename(g).split('_')[0][4:]
    # print(pos)
    df = pd.read_csv(g, sep='\t')
    df.columns = ['cell_idx', 'state_idx']
    all_df[pos] = df

In [84]:
all_df.keys()

dict_keys(['78', '4', '3', '910', '6', '5', '2'])

In [36]:
all_df['78']

,cell_idx,state_idx
0,3,3
1,4,3
2,8,3
3,10,3
4,11,3
5,18,3
6,39,2
7,48,2
8,49,2
9,53,3


In [46]:
type(all_df['78'].loc[all_df['78'].index[all_df['78']['cell_idx'] == 3].tolist()[0], 'state_idx'])

numpy.int64

In [177]:
# prune any cells which we don't have state for

baseMem_totalcost = []
laml2_totalcost = []

for pos in [2, 3, 4, 5, 6, 78, 910]:
# pos = 2
    print("colony", pos)
    laml_tfile = f'{laml2_folder_path}/LAML2_p{pos}_baseMem_trees.nwk'
    laml_tree = treeswift.read_tree_newick(laml_tfile)

    leaf_label_df = all_df[str(pos)]
    states = [1, 2, 3, 4, 5]

    laml2_totalcost.append(compute_parsimony_cost(laml_tree, leaf_label_df, states))

    baseMem_tfile = f'/Users/gc3045/laml2_experiments/proc_realdata/baseMemoir/p{pos}_starting_tree.nwk'
    baseMem_tree = treeswift.read_tree_newick(baseMem_tfile)

    baseMem_totalcost.append(compute_parsimony_cost(baseMem_tree, leaf_label_df, states))
    

colony 2
colony 3
colony 4
colony 5
colony 6
colony 78
colony 910


In [179]:
import numpy as np


print("laml26", np.sum(np.array(laml2_totalcost)))
print("baseMem", np.sum(np.array(baseMem_totalcost)))

laml2 73
baseMem 59
